In [6]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import datetime
from tickers import getTickers
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [72]:
class StockDataset(Dataset):
    def __init__(self, directory, ticker, context = 5) -> None:
        super().__init__()
        self.info = []
        self.context = context

        self.info = pd.read_csv(f"{directory}/{ticker}.csv")

        self.info = np.array(self.info.values.tolist())
        self.feature_peaks = {}
                
        if len(self.info) > 1:
            for feature_col in range(1, len(self.info[0,:])):
                col = self.info[:,feature_col].astype(np.double)
                high, low, col = self.norm(col)
                self.info[:,feature_col] = col
                self.feature_peaks[feature_col] = (low, high)

    def __len__(self):
        return len(self.info) - self.context if len(self.info) > self.context else len(self.info)
    def __getitem__(self, index):
        index+=self.context
        res = []

        for day in self.info[index-self.context: index]:
            time = self.transformDay(day[0])
            res.append(np.concatenate((day[1:].astype(np.double) , time), axis = 0))
            # res.append(day[1:])

        
        previousDays = torch.Tensor(res)
        closingPrice = torch.tensor(float(self.info[index][4])).float()
        return previousDays, closingPrice
    def transformDay(self, date):

        dt = datetime.datetime.strptime(date, "%Y-%m-%d")
        res = [dt.weekday(), dt.day, dt.month]
        return np.array(res)
    def norm(self, feature):
        high, low = max(feature), min(feature)
        for i in range(len(feature)):
            feature[i] = round(((feature[i]-low)/(high-low)), 4)
        return high, low, feature
    def reverse(self, output):
        return ((output)*(self.feature_peaks[4][1]-self.feature_peaks[4][0])) + self.feature_peaks[4][0]
    def getLatest(self):
        res = []

        for day in self.info[len(self.info)-self.context: len(self.info)]:
            time = self.transformDay(day[0])
            res.append(np.concatenate((day[1:].astype(np.double) , time), axis = 0))
            # res.append(day[1:])

        
        previousDays = torch.Tensor(res)
        return previousDays


dkng = StockDataset("data", "AAPL")
print(dkng[0], dkng[3010])



(tensor([[4.3000e-03, 3.8000e-03, 4.7000e-03, 4.5000e-03, 3.9000e-03, 2.4600e-01,
         0.0000e+00, 4.0000e+00, 1.0000e+00],
        [4.6000e-03, 4.0000e-03, 4.9000e-03, 4.6000e-03, 3.9000e-03, 3.0480e-01,
         1.0000e+00, 5.0000e+00, 1.0000e+00],
        [4.5000e-03, 3.9000e-03, 4.3000e-03, 3.9000e-03, 3.3000e-03, 2.7780e-01,
         2.0000e+00, 6.0000e+00, 1.0000e+00],
        [4.0000e-03, 3.3000e-03, 4.0000e-03, 3.8000e-03, 3.3000e-03, 2.3700e-01,
         3.0000e+00, 7.0000e+00, 1.0000e+00],
        [3.7000e-03, 3.3000e-03, 4.0000e-03, 4.1000e-03, 3.5000e-03, 2.2100e-01,
         4.0000e+00, 8.0000e+00, 1.0000e+00]]), tensor(0.0037)) (tensor([[ 0.9894,  0.9943,  0.9717,  0.9583,  0.9586,  0.0593,  3.0000, 16.0000,
         12.0000],
        [ 0.9358,  0.9506,  0.9654,  0.9519,  0.9521,  0.0839,  4.0000, 17.0000,
         12.0000],
        [ 0.9263,  0.9340,  0.9522,  0.9438,  0.9441,  0.0361,  0.0000, 20.0000,
         12.0000],
        [ 0.9451,  0.9490,  0.9620,  0.9626, 

In [92]:
t = dkng[10][0]
t = t.unsqueeze(0)
print(t.shape)
rnn = nn.GRU(9, hidden_size = 10, bias = False)
o, h = rnn(t)

print(o.shape, h.shape)

for i, j in enumerate(dev):
    if i > 100: break
    print(j)

torch.Size([1, 5, 9])
torch.Size([1, 5, 10]) torch.Size([1, 5, 10])
(tensor([[7.6225e+00, 7.6607e+00, 7.5850e+00, 7.6432e+00, 6.5530e+00, 4.9373e+08,
         0.0000e+00, 4.0000e+00, 1.0000e+00],
        [7.6643e+00, 7.6996e+00, 7.6161e+00, 7.6564e+00, 6.5644e+00, 6.0190e+08,
         1.0000e+00, 5.0000e+00, 1.0000e+00],
        [7.6564e+00, 7.6868e+00, 7.5268e+00, 7.5346e+00, 6.4599e+00, 5.5216e+08,
         2.0000e+00, 6.0000e+00, 1.0000e+00],
        [7.5625e+00, 7.5714e+00, 7.4661e+00, 7.5207e+00, 6.4480e+00, 4.7713e+08,
         3.0000e+00, 7.0000e+00, 1.0000e+00],
        [7.5107e+00, 7.5714e+00, 7.4664e+00, 7.5707e+00, 6.4909e+00, 4.4761e+08,
         4.0000e+00, 8.0000e+00, 1.0000e+00]]), tensor(7.5039))
(tensor([[7.6643e+00, 7.6996e+00, 7.6161e+00, 7.6564e+00, 6.5644e+00, 6.0190e+08,
         1.0000e+00, 5.0000e+00, 1.0000e+00],
        [7.6564e+00, 7.6868e+00, 7.5268e+00, 7.5346e+00, 6.4599e+00, 5.5216e+08,
         2.0000e+00, 6.0000e+00, 1.0000e+00],
        [7.5625e+00, 7.

In [3]:
class Encoder(nn.Module):
    def __init__(self, num_hidden = 128, num_layers = 1, context = 5, features = 9, dropout = 0.1) -> None:
        super().__init__()
        self.rnn = nn.GRU(features, num_hidden, num_layers, batch_first = True)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.dropout(x)
        output, state = self.rnn(x)
        return output, state

class PlainDecoder(nn.Module):
    def __init__(self, num_hidden = 128, num_layers = 1, context = 5, features = 9, dropout = 0.1) -> None:
        super().__init__()
        self.rnn = nn.GRU(features, num_hidden, num_layers, batch_first = True)
        self.dense = nn.Linear(num_hidden, 1)
        self.dropout = nn.Dropout(dropout)


    def forward(self, x, state):

        # x = self.dropout(x)
        output, state = self.rnn(x, state)
        output = self.dense(output)
        return output

class Seq2Seq(nn.Module):
    def __init__(self, num_hidden = 128, num_layers = 1, context = 5, features = 9, dropout = 0.1) -> None:
        super().__init__()
        self.encoder = Encoder()
        self.decoder = PlainDecoder()
    def forward(self, x):
        # print(x.shape)
        encoder_output, encoder_hidden = self.encoder(x)# hidden --> layer, batch, hidden dim
        most_recent_day = x[:,-1,:].unsqueeze(1) #batch, days, features
        output = self.decoder(most_recent_day, encoder_hidden).squeeze(2)
        return output


class PlainModel(nn.Module):
    def __init__(self, num_hidden = 128, num_layers = 1, context = 5, features = 9, dropout = 0.1) -> None:
        super().__init__()
        self.rnn = nn.GRU(features, num_hidden, num_layers, batch_first = True)
        self.dense = nn.Linear(num_hidden, 1)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.dropout(x)
        output, state = self.rnn(x)
        output = self.dense(output)
        return output[:,-1]


In [40]:
model = Seq2Seq().to(device)
# train_loader = DataLoader(dev, batch_size = 10, shuffle= True)
optimizer = optim.Adam(model.parameters(), lr = 0.001)
loss = nn.MSELoss()
epochs = 10

for epoch in range(epochs):
    total_mse = 0
    for train_loader in train:
        for x, y in train_loader:
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = model(x).squeeze()
        
            batch_loss = loss(output, y)
            total_mse+=batch_loss
            batch_loss.backward()
            optimizer.step()
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, total_mse))   

Epoch [1/10], Loss: 24.9930
Epoch [2/10], Loss: 18.4269
Epoch [3/10], Loss: 18.0211
Epoch [4/10], Loss: 17.8454
Epoch [5/10], Loss: 17.6108
Epoch [6/10], Loss: 17.5327
Epoch [7/10], Loss: 17.5647
Epoch [8/10], Loss: 17.3212
Epoch [9/10], Loss: 17.2589
Epoch [10/10], Loss: 17.2402


In [21]:
from itertools import chain
dev_tickers = ['AAPL','TSLA','COKE', 'AMAT']
dev = [StockDataset("data", ticker) for ticker in dev_tickers]
dev = [DataLoader(ticker, batch_size = 64, shuffle= True) for ticker in dev]
# dev = list(chain.from_iterable(dev))
# print(len(dev))

In [38]:
train_tickers = getTickers()
train = [StockDataset("data", ticker) for ticker in train_tickers]
train = [obj for obj in train if len(obj) == 3011]
print(f"Training set has {len(train)} tickers")
train = [DataLoader(ticker, batch_size = 64, shuffle= True) for ticker in train]

test = train[:20]
train = train[20:]

<ipython-input-37-8d4ef4ea189d>:42: RuntimeWarning: invalid value encountered in double_scalars
  feature[i] = round(((feature[i]-low)/(high-low)), 4)


Training set has 1502 tickers


In [41]:
model2 = PlainModel(features=9).to(device)
# train_loader = DataLoader(dev, batch_size = 10, shuffle= True)
optimizer = optim.Adam(model2.parameters(), lr = 0.001)
loss = nn.MSELoss()
epochs = 10

for epoch in range(epochs):
    total_mse = 0
    for train_loader in train:
        for x, y in train_loader:
            x = x.to(device)
            y = y.to(device)
            optimizer.zero_grad()
            output = model2(x).squeeze()
        
            batch_loss = loss(output, y)
            total_mse+=batch_loss
            batch_loss.backward()
            optimizer.step()
            # break
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, epochs, total_mse))   
    


Epoch [1/10], Loss: 37.5448
Epoch [2/10], Loss: 20.4773
Epoch [3/10], Loss: 19.4226
Epoch [4/10], Loss: 18.8931
Epoch [5/10], Loss: 18.6525
Epoch [6/10], Loss: 18.3881
Epoch [7/10], Loss: 18.3717
Epoch [8/10], Loss: 18.1599
Epoch [9/10], Loss: 18.2083
Epoch [10/10], Loss: 18.1048


In [46]:
s = dkng[100]
x, y = s[0], s[1]
x = x.to(device)
y = y.to(device)
print(x[0],y)
x = x.unsqueeze(0)
o = model(x)

print(o)
print(dkng.reverse(o.item()))
print(dkng.reverse(y.item()))

tensor([1.1900e-02, 1.1800e-02, 1.2500e-02, 1.2700e-02, 1.0800e-02, 3.3980e-01,
        3.0000e+00, 2.7000e+01, 5.0000e+00], device='cuda:0') tensor(0.0132, device='cuda:0')
tensor([[0.0186]], device='cuda:0', grad_fn=<SqueezeBackward1>)
10.06372883224228
9.137131085255325


In [55]:
test = 'ZUMZ'
test = StockDataset(test)
sample = test[len(test)-1]
x, y =sample[0], sample[1]
x = x.to(device)
y = y.to(device)
x = x.unsqueeze(0)
o = model(x)

print(o)
x = x.squeeze(0)
daysPrior = [test.reverse(day[4]) for day in x]
print(daysPrior)
print(test.reverse(o.item()))
print(test.reverse(y.item()))

tensor([[0.7732]], device='cuda:0', grad_fn=<SqueezeBackward1>)
[tensor(45.0009, device='cuda:0'), tensor(44.7806, device='cuda:0'), tensor(42.6087, device='cuda:0'), tensor(44.4611, device='cuda:0'), tensor(45.1218, device='cuda:0')]
44.83547083447195
45.8990049913444


# Day To Day Predictions

In [73]:
import time
import requests
import os
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
if not os.path.exists('predict/'):
    os.mkdir('predict/')
checking = "DKNG"

today = int(time.time())
endPoint = 1638316800 #earlier date

url =f'https://query1.finance.yahoo.com/v7/finance/download/\
{checking}?period1={endPoint}&period2={today}&interval=1d&events=history&includeAdjustedClose=true'

try:
    res = requests.get(url, headers=headers, allow_redirects=True)
    open(f'predict/{checking}.csv', 'wb').write(res.content)
except:
    pass
    # print("Error Grabbing {}".format(ticker))
    
data = StockDataset("predict", checking)

predict = data.getLatest()
x=predict
#take off gpu when forreal
x = x.to(device)


x = x.unsqueeze(0)
o = model(x)
print(data.reverse(o.item()))
# if os.path.exists(f"predict/{checking}.csv"):
#     os.remove(f"predict/{checking}.csv")

27.63799475025891
